In [1]:
import pandas as pd
import os
import re

In [2]:
def parse_log_file(log_file_path):
    model_size = None
    inference_time = None
    
    with open(log_file_path, 'r') as file:
        for line in file:
            # Extract model size
            size_match = re.search(r'Size \(MB\): ([\d\.]+)', line)
            if size_match:
                model_size = float(size_match.group(1))
            
            # Extract average inference time
            time_match = re.search(r'Average inference time: ([\d\.]+)', line)
            if time_match:
                inference_time = float(time_match.group(1))
                
    return model_size, inference_time

In [6]:
df = pd.read_csv('../../analysis/experiment_values.csv')
df.head()

,Task,Compression Method,Parameter,Value
0,Defect Prediction,No One,Avg Inference Time CUDA,0.010
1,Defect Prediction,Distillation (distilbert),Avg Inference Time CUDA,0.005
2,Defect Prediction,Quantization (quanto-qfloat8),Avg Inference Time CUDA,0.021
3,Defect Prediction,Quantization (quanto-qint8),Avg Inference Time CUDA,0.019
4,Defect Prediction,Quantization (quanto-qint4),Avg Inference Time CUDA,0.030


In [7]:
for logs in os.listdir('logs_graph'):
  model_size, inf_time = parse_log_file(f'logs_graph/{logs}')
  task = 'Defect Prediction Graph'
  if 'prune4' in logs:
    compression = 'Pruning 0.4'
  elif 'prune6' in logs:
    compression = 'Pruning 0.6'
  elif 'prune' in logs:
    compression = 'Pruning 0.2'
  elif 'quantf8' in logs:
    compression = 'Quantization (quanto-qfloat8)'
  elif 'quant4' in logs:
    compression = 'Quantization (quanto-qint4)'
  elif 'quant' in logs:
    compression = 'Quantization (quanto-qint8)'
  else:
    compression = 'No One'
  if 'cuda' in logs:
    df = pd.concat([df, pd.DataFrame({
      'Task': [task], 'Compression Method': [compression], 'Parameter': ["Avg Inference Time CUDA"], 'Value': [inf_time]})])  
  else:
    df = pd.concat([df, pd.DataFrame({
      'Task': [task], 'Compression Method': [compression], 'Parameter': ["Avg Inference Time CPU"], 'Value': [inf_time]})])
    df = pd.concat([df, pd.DataFrame({
      'Task': [task], 'Compression Method': [compression], 'Parameter': ["Model Size"], 'Value': [model_size]})])
 

In [8]:
df.tail(n=25)

,Task,Compression Method,Parameter,Value
238,Code Search Graph,Quantization (quanto-qint8),MRR@5,0.343100
239,Code Search Graph,No One,MRR,0.361100
240,Code Search Graph,No One,MRR@1,0.269400
241,Code Search Graph,No One,MRR@5,0.341900
242,Code Search Graph,Quantization (quanto-qfloat8),MRR,0.361100
243,Code Search Graph,Quantization (quanto-qfloat8),MRR@1,0.270000
244,Code Search Graph,Quantization (quanto-qfloat8),MRR@5,0.341900
0,Defect Prediction Graph,Pruning 0.6,Avg Inference Time CUDA,0.010251
0,Defect Prediction Graph,Pruning 0.2,Avg Inference Time CPU,47.496147
0,Defect Prediction Graph,Pruning 0.2,Model Size,498.655532


In [9]:
df.to_csv('../../analysis/experiment_values.csv', index=False)